<a href="https://colab.research.google.com/github/gustavosr8/SpiroPhone/blob/master/Espirometria_regressionNG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import librosa
import librosa.display

import scipy

from scipy import signal
from scipy.signal import butter, filtfilt

import sklearn
from sklearn import preprocessing, decomposition, model_selection, ensemble, metrics, pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.model_selection import RandomizedSearchCV, train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import precision_score, accuracy_score, explained_variance_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Activation

!pip install scikeras
from scikeras.wrappers import KerasRegressor


#Leitura dos dados



In [2]:
drive.mount('/content/drive')
path='/content/drive/MyDrive/IC/codigo/assets/'
dataset = 'datasetng.csv'

df = pd.read_csv(path+dataset)
df.head()

# Checando se arquivos existem, numero de canais e taxa de amostragem
n_files = 0
missing_files = 0
taxas_de_amostragem = []
duracoes = []
amostras = []
classe = []
for index, row in df.iterrows():
  full_fname = path + str(row['REGISTRO'])+'.wav'
  if (n_files % 10 == 0):
    print(str(n_files+1) + " " + full_fname)
  if not os.path.exists(full_fname):
    print(str(row['REGISTRO']) + " (index=" + str(n_files) + ") does not exist!")
    df.drop(df.index[n_files])
    missing_files += 1
    n_files += 1
  else:
    y, sr = librosa.load(full_fname, sr=None, mono=True)
    taxas_de_amostragem.append(sr)
    duracoes.append(len(y)/sr)
    amostras.append(y)
    n_files += 1
print("Encontrei esse numero de arquivos: " + str(n_files-missing_files))
df['sr'] = taxas_de_amostragem
df['len'] = duracoes

teste1, sr = librosa.load(path+'10509684.wav', sr=None, mono=True)
teste2, sr = librosa.load(path+'10523145.wav', sr=None, mono=True)

Mounted at /content/drive
1 /content/drive/MyDrive/IC/codigo/assets/4306486.wav
11 /content/drive/MyDrive/IC/codigo/assets/10348393.wav
21 /content/drive/MyDrive/IC/codigo/assets/4340095.wav
31 /content/drive/MyDrive/IC/codigo/assets/10305974n.wav
41 /content/drive/MyDrive/IC/codigo/assets/863418n.wav
51 /content/drive/MyDrive/IC/codigo/assets/10425372g.wav
61 /content/drive/MyDrive/IC/codigo/assets/893629g.wav
Encontrei esse numero de arquivos: 69


In [3]:
df

,REGISTRO,SEXO,IDADE,PESO,ALTURA,CVF,CVF%,VEF1,VEF1%,VEF1/CVF,VEF1/CVF%,FEF,FEF%,PF,PF%,DISTÚRBIO,sr,len
0,4306486,F,71,76.9,160.0,2.58,92,1.63,76,0.63,81,1.17,57,4.30,74,OBSTRUÇÃO,48000,11.677125
1,10429556,M,63,91.6,187.5,4.33,85,2.52,63,0.58,74,2.05,58,7.21,72,OBSTRUÇÃO,48000,12.490104
2,3724996,F,65,58.5,162.0,2.50,83,1.77,75,0.71,90,1.21,55,4.84,80,OBSTRUÇÃO,48000,17.440042
3,10433358,F,45,61.6,146.0,2.12,80,1.16,52,0.54,66,0.50,20,2.91,50,OBSTRUÇÃO,48000,14.128250
4,10425372,F,56,65.8,149.0,1.48,55,0.58,26,0.39,47,0.19,8,2.02,31,OBSTRUÇÃO,48000,14.616604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,4569469g,M,81,81.1,166.0,2.14,57,1.82,65,0.85,113,2.34,94,5.57,55,RESTRIÇÃO,48000,10.131042
65,4027607g,F,50,101.8,159.0,1.90,61,1.68,66,0.88,109,2.88,113,7.44,118,RESTRIÇÃO,48000,19.102667
66,4340095g,F,28,77.0,168.0,4.30,107,3.21,96,0.75,89,2.57,70,6.31,77,OBSTRUÇÃO,48000,6.894146
67,10462710g,F,52,75.4,155.0,2.42,83,2.04,86,0.84,103,2.57,107,7.20,119,NORMAL,48000,7.130063


#Processamento de Features

##Pressão e fluxo de ar aproximados

A pressão proporcional de ar de saída é dada por uma função de transferência do áudio original.

In [4]:
def butter_lowpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / (0.5*fs)
    # Pega os coeficientes do filtro
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

def butter_highpass_filter(data, cutoff, fs, order):
    normal_cutoff = cutoff / (0.5*fs)
    # Pega os coeficientes do filtro
    b, a = butter(order, normal_cutoff, btype='highpass', analog=False)
    y = filtfilt(b, a, data)
    return y

# Requerimentos do filtro
fs = sr         # sample rate, Hz
cutoff = 1000   # Frequência de corte do filtro, em Hz
order = 5       # Ordem do filtro

plips=[]
for i in amostras:
  plips.append(butter_highpass_filter(i, cutoff, fs, order))

rlips=  0.05                                    #raio de abertura da boca de 5cm
ulips=[]

for p in plips:
  ulips.append(2*np.pi*rlips**2*np.sqrt(2*abs(p)))   #proporcional ao fluxo de ar saindo da boca

amplitude_envelope_plips = []
amplitude_envelope_ulips = []

for i in range(len(plips)):
  amplitude_envelope_plips.append(librosa.feature.rms(y=plips[i])[0,:])
  amplitude_envelope_ulips.append(librosa.feature.rms(y=ulips[i])[0,:])

In [5]:
def descritores(ulips, plips, df_row ):
  return np.array([np.max(ulips), np.sum(ulips), np.sum(ulips[0:df_row['sr']]), df_row['IDADE'], df_row['PESO'], df_row['ALTURA']])

n_files = 0
vetores = []
for i in range(len(ulips)):
  vetores.append(descritores(amplitude_envelope_ulips[i], amplitude_envelope_plips[i], df.iloc[i]))
  n_files +=1
print("Processei esse número de arquivos: " + str(n_files))
vetores = np.array(vetores)
print(vetores.shape)

scaler = StandardScaler()
scaler.fit(vetores)
dados = scaler.transform(vetores)
params = ['CVF', 'VEF1','PF']


Processei esse número de arquivos: 69
(69, 6)


In [6]:
#Dados para teste final

plips_test=[butter_highpass_filter(teste1, cutoff, fs, order),
            butter_highpass_filter(teste2, cutoff, fs, order)]

ulips_test=[]
for p in plips_test:
  ulips_test.append(2*np.pi*rlips**2*np.sqrt(2*abs(p)))   #proporcional ao fluxo de ar saindo da boca

amplitude_envelope_plips_test = []
amplitude_envelope_ulips_test = []

for i in range(len(plips_test)):
  amplitude_envelope_plips_test.append(librosa.feature.rms(y=plips_test[i])[0,:])
  amplitude_envelope_ulips_test.append(librosa.feature.rms(y=ulips_test[i])[0,:])

vetores_test = [np.array([np.max(amplitude_envelope_ulips[0]), np.sum(amplitude_envelope_ulips[0]), np.sum(amplitude_envelope_ulips[0][0:48000]), 55, 78.6, 158]),
                np.array([np.max(amplitude_envelope_ulips[1]), np.sum(amplitude_envelope_ulips[1]), np.sum(amplitude_envelope_ulips[1][0:48000]), 66, 78.6, 171])]
scaler = StandardScaler()
scaler.fit(vetores_test)
dados_test = scaler.transform(vetores_test)

#Random forest regressor

In [7]:
def RandomForest(param):
  X = dados
  y = df[param].to_numpy()


  # create regressor object
  regressor = RandomForestRegressor(n_estimators=1000, random_state=42, verbose=0, n_jobs=-1, oob_score=True)

  scores = cross_val_score(regressor, X, y, cv=5, scoring='explained_variance')

  # fit the regressor with x and y data
  regressor.fit(X, y)

  y_pred = regressor.predict(dados_test)
  print(param)
  print("predicted:")
  print(y_pred)
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")

for param in params:
  RandomForest(param)

CVF
predicted:
[2.42986 3.06979]
0.97 accuracy with a standard deviation of 0.01
.
.
.
VEF1
predicted:
[2.34514 2.53752]
0.96 accuracy with a standard deviation of 0.02
.
.
.
PF
predicted:
[7.0586  7.89719]
0.94 accuracy with a standard deviation of 0.05
.
.
.


#Regressor linear


In [8]:
def RegressorLinear(param):
  X = dados
  y = df[param].to_numpy()

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  # create regressor object
  model = Sequential()
  # Define the model consisting of a single neuron.
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)


  scores = []
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X[train], y[train], epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))

  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test, verbose = 0)

  print(param)
  print("predicted:")
  print(y_pred)
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  RegressorLinear(param)

CVF
predicted:
[[2.8462625]
 [3.0357919]]
0.75 accuracy with a standard deviation of 0.07
.
.
.
VEF1
predicted:
[[2.2598727]
 [1.7420794]]
0.54 accuracy with a standard deviation of 0.13
.
.
.
PF
predicted:
[[6.333289]
 [4.951503]]
0.27 accuracy with a standard deviation of 0.40
.
.
.


In [9]:
def MLP(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  model = Sequential()
  # create regressor object
  model.add(Dense(3, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
  model.add(Dense(6, kernel_initializer='normal'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)

  scores = []
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X_train, y_train, epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))
  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test)

  print(param)
  print("predicted:")
  print(y_pred)
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  MLP(param)


1/1 [==============================] - 0s 34ms/step
CVF
predicted:
[[2.6256428]
 [3.1218412]]
0.85 accuracy with a standard deviation of 0.06
.
.
.
1/1 [==============================] - 0s 20ms/step
VEF1
predicted:
[[2.2501504]
 [1.9692864]]
0.75 accuracy with a standard deviation of 0.11
.
.
.
1/1 [==============================] - 0s 20ms/step
PF
predicted:
[[5.97674  ]
 [4.7278543]]
0.49 accuracy with a standard deviation of 0.29
.
.
.


In [10]:
def MLP(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  model = Sequential()
  # create regressor object
  model.add(Dense(6, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
  model.add(Dense(3, kernel_initializer='normal'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)

  scores = []
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X_train, y_train, epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))
  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test)

  print(param)
  print("predicted:")
  print(y_pred)
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  MLP(param)


1/1 [==============================] - 0s 52ms/step
CVF
predicted:
[[2.4064395]
 [3.2315037]]
0.93 accuracy with a standard deviation of 0.03
.
.
.
1/1 [==============================] - 0s 20ms/step
VEF1
predicted:
[[1.9805009]
 [1.7526621]]
0.75 accuracy with a standard deviation of 0.11
.
.
.
1/1 [==============================] - 0s 19ms/step
PF
predicted:
[[6.1966  ]
 [4.434329]]
0.50 accuracy with a standard deviation of 0.25
.
.
.


In [11]:
def MLP(param):
  X = dados
  y = df[param].to_numpy()

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Fix seed to make training deterministic.
  SEED_VALUE = 42
  np.random.seed(SEED_VALUE)
  tf.random.set_seed(SEED_VALUE)

  model = Sequential()
  # create regressor object
  model.add(Dense(10, input_shape=(X_train.shape[1],), kernel_initializer='normal', activation='relu'))
  model.add(Dense(8, kernel_initializer='normal'))
  model.add(Dense(6, kernel_initializer='normal'))
  model.add(Dense(4, kernel_initializer='normal'))
  model.add(Dense(2, kernel_initializer='normal'))
  model.add(Dense(1, kernel_initializer='normal'))
  model.add(Dense(units=1, input_shape=(X.shape[1],)))
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.005), loss='mse')
  es = EarlyStopping(monitor='loss', mode='min', verbose=0, patience=20, restore_best_weights=True)

  scores = []
  fold_no = 1
  kfold = KFold(n_splits=5, shuffle=True)
  for train, test in kfold.split(X, y):

    model.fit(X_train, y_train, epochs=1000, callbacks = es, verbose = 0)
    y_pred = model.predict(X[test], verbose = 0)
    scores.append(explained_variance_score(y[test],y_pred))
  scores = np.array(scores)
  model.fit(X, y, epochs=1000, callbacks = es, verbose = 0)
  y_pred = model.predict(dados_test)

  print(param)
  print("predicted:")
  print(y_pred)
  print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
  print(".\n.\n.")
for param in params:
  MLP(param)


1/1 [==============================] - 0s 21ms/step
CVF
predicted:
[[2.565638 ]
 [3.2633638]]
0.90 accuracy with a standard deviation of 0.04
.
.
.
1/1 [==============================] - 0s 20ms/step
VEF1
predicted:
[[1.7851597]
 [1.6987199]]
0.75 accuracy with a standard deviation of 0.12
.
.
.
1/1 [==============================] - 0s 34ms/step
PF
predicted:
[[5.6732306]
 [4.161857 ]]
0.69 accuracy with a standard deviation of 0.10
.
.
.
